In [33]:
# Bokeh Libraries
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CategoricalColorMapper, Div, NumeralTickFormatter
from bokeh.layouts import gridplot, column

# Import data
from read_nba_data import team_stats

# Out to file
output_notebook()

Loading BokehJS ...

In [23]:
# Prepare data
gm_stats = (team_stats[team_stats['teamAbbr'] == 'CHI']
                .loc[:, ['gmDate',
                         'teamPTS',
                         'teamTRB',
                         'teamAST',
                         'teamTO',
                         'team2P%',
                         'team3P%',
                         'opptPTS']]
                .sort_values('gmDate'))

# Add game number
gm_stats['game_num'] = range(1, len(gm_stats)+1)

# Make a win_loss column
win_loss = []
for _, row in gm_stats.iterrows():

    if row['teamPTS'] > row['opptPTS']:
        win_loss.append('W')
    else:
        win_loss.append('L')

# Add the win_loss data to the DataFrame
gm_stats['winLoss'] = win_loss

In [28]:
# Store the data in a ColumnDataSource
gm_stats_cds = ColumnDataSource(gm_stats)

# Create a CategoricalColorMapper that assigns a color to wins and losses
win_loss_mapper = CategoricalColorMapper(factors = ['W', 'L'],
                                         palette=['green', 'red'])


# Map stat names to columns in the data
stat_names = {'Points': 'teamPTS',
              'Assists': 'teamAST',
              'Rebounds': 'teamTRB',
              'Turnovers': 'teamTO'}

# The figure for each stat will be held in this dict
stat_figs = {}

# For each stat in the dict
for stat_label, stat_col in stat_names.items():

    # Create a figure
    fig = figure(y_axis_label=stat_label,
                 height=200, width=400,
                 x_range=(1, 10), tools=['xpan', 'reset', 'save'])

    # Configure vbar
    fig.vbar(x='game_num', top=stat_col, source=gm_stats_cds, width=0.9,
             color=dict(field='winLoss', transform=win_loss_mapper))

    # Add the figure to stats_figs dict
    stat_figs[stat_label] = fig

# Create layout
grid = gridplot([[stat_figs['Points'], stat_figs['Assists']],
                [stat_figs['Rebounds'], stat_figs['Turnovers']]])

# Link together the x-axes
stat_figs['Points'].x_range = \
    stat_figs['Assists'].x_range = \
    stat_figs['Rebounds'].x_range = \
    stat_figs['Turnovers'].x_range

# Add a title for the entire visualization using Div
html = """<h3>Chicago Game Log</h3>
<b><i>2017-18 </i></b>
<br>
<i>Wins (green) vs Losses (red)</i>
"""
sup_title = Div(text=html)

# Visualize
show(column(sup_title, grid))

In [34]:
# Specify the tools
toolList = ['lasso_select', 'tap', 'reset', 'save']

# Create a figure relating the percentages
pctFig = figure(title='2PT FG % vs 3PT FG %, 2017-18 Regular Season',
                height=400, width=400, tools=toolList,
                x_axis_label='2PT FG%', y_axis_label='3PT FG%')

# Draw with circle markers
pctFig.circle(x='team2P%', y='team3P%', source=gm_stats_cds,
              size=12, color='black')

# Format the y-axis and x-axis tick labels as percentages
pctFig.xaxis[0].formatter = NumeralTickFormatter(format='00.0%')
pctFig.yaxis[0].formatter = NumeralTickFormatter(format='00.0%')

# Create a figure relating the totals
totFig = figure(title='Team Points vs Opponent Points, 2017-18 Regular Season',
                height=400, width=400, tools=toolList,
                x_axis_label='Team Points', y_axis_label='Opponent Points')

# Draw with square markers
totFig.square(x='teamPTS', y='opptPTS', source=gm_stats_cds, size=10,
              color=dict(field='winLoss', transform=win_loss_mapper))

# Create layout
grid = gridplot([[pctFig, totFig]])

# Visualize
show(grid)